# In this project I will be using IMDB reviews and based on the word sentiments, the texts will be classified as either positive and negative

### Import all the libraries and the datasets

In [23]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

In [6]:
#Load the imdb reviews along with metadata
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete71FRGN/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete71FRGN/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete71FRGN/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [7]:
print(info)

tfds.core.DatasetInfo(
    name='imdb_reviews',
    full_name='imdb_reviews/plain_text/1.0.0',
    description="""
    Large Movie Review Dataset. This is a dataset for binary sentiment
    classification containing substantially more data than previous benchmark
    datasets. We provide a set of 25,000 highly polar movie reviews for training,
    and 25,000 for testing. There is additional unlabeled data for use as well.
    """,
    config_description="""
    Plain text
    """,
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    data_dir=PosixGPath('/tmp/tmptsco3nf7tfds'),
    file_format=tfrecord,
    download_size=80.23 MiB,
    dataset_size=129.83 MiB,
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
        'text': Text(shape=(), dtype=string),
    }),
    supervised_keys=('text', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=25000, num_shards=1>,
        'train': <SplitInfo

In [8]:
# Print the contents of the dataset
print(imdb)

{Split('train'): <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>, Split('test'): <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>, Split('unsupervised'): <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}


In [16]:
# Print 5 examples of the text and labels
for example,label in imdb['train'].take(5):
  print(example)
  print(label,'\n')

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
tf.Tensor(0, shape=(), dtype=int64) 

tf.Tensor(b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on

In [22]:
# Put the texts and labels to variables for the preprocessing

training_texts = []
training_labels = []

testing_texts = []
testing_labels = []

for s,l in imdb['train']:
  training_texts.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())


for s,l in imdb['test']:
  testing_texts.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())

training_labels = np.array(training_labels)
testing_labels = np.array(testing_labels)

### Generate padded sequences using tokeninzer and paddings

In [28]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# First initialize the parameters
vocab_size = 12000
oov_tok = "<OOV>"
trun_type = "post"
max_length = 125
embedding_dim = 16

# Instantiate a tokenizer and get the word index
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_texts)
word_index = tokenizer.word_index

# Generate and pad the training sentences
sequences = tokenizer.texts_to_sequences(training_texts)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trun_type)

# Generate and pad the testing sentences
test_sequences = tokenizer.texts_to_sequences(testing_texts)
test_padded = pad_sequences(test_sequences, maxlen=max_length, truncating=trun_type)


### Build and compile the model

In [33]:
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Flatten(),
    keras.layers.Dense(6, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

# Compile the model
model.compile(loss=keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

# Get the model summary
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 125, 16)           192000    
                                                                 
 flatten_4 (Flatten)         (None, 2000)              0         
                                                                 
 dense_8 (Dense)             (None, 6)                 12006     
                                                                 
 dense_9 (Dense)             (None, 1)                 7         
                                                                 
Total params: 204013 (796.93 KB)
Trainable params: 204013 (796.93 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


### Train the model

In [34]:
num_epoch = 10

model.fit(padded, training_labels, epochs=num_epoch, validation_data=(test_padded, testing_labels))

Epoch 1/10
782/782 [==============================] - 8s 9ms/step - loss: 0.4852 - accuracy: 0.7508 - val_loss: 0.3737 - val_accuracy: 0.8343
Epoch 2/10
782/782 [==============================] - 9s 11ms/step - loss: 0.2245 - accuracy: 0.9160 - val_loss: 0.4176 - val_accuracy: 0.8182
Epoch 3/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0788 - accuracy: 0.9813 - val_loss: 0.4997 - val_accuracy: 0.8137
Epoch 4/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0224 - accuracy: 0.9968 - val_loss: 0.5857 - val_accuracy: 0.8087
Epoch 5/10
782/782 [==============================] - 6s 7ms/step - loss: 0.0080 - accuracy: 0.9991 - val_loss: 0.6508 - val_accuracy: 0.8067
Epoch 6/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0029 - accuracy: 0.9998 - val_loss: 0.7115 - val_accuracy: 0.8093
Epoch 7/10
782/782 [==============================] - 6s 7ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.7598 - val_accuracy: 0.8092
Epoch

### Observe the weights of the embedded layer

In [43]:
# Get the embedding layer of the model
emd_layer = model.layers[0]

#Get the weights of the layer
weights = emd_layer.get_weights()[0]

weights.shape

(12000, 16)

### Visualize the word polarization on TensorFlow

In [44]:
# Get the index-word dictionary
reverse_word_index = tokenizer.index_word

import io

# Open writeable files
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

# Initialize the loop. Start counting at `1` because `0` is just for the padding
for word_num in range(1, vocab_size):

  # Get the word associated at the current index
  word_name = reverse_word_index[word_num]

  # Get the embedding weights associated with the current index
  word_embedding = weights[word_num]

  # Write the word name
  out_m.write(word_name + "\n")

  # Write the word embedding
  out_v.write('\t'.join([str(x) for x in word_embedding]) + "\n")

# Close the files
out_v.close()
out_m.close()

# Import files utilities in Colab
try:
  from google.colab import files
except ImportError:
  pass

# Download the files
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>